<a href="https://colab.research.google.com/github/RavindraGR7/BIG-FIX/blob/main/final_project_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

# Load the dataset
monthly_data = pd.read_csv('monthly_data.csv', parse_dates=['Date'])


In [3]:
# Calculate the Sharpe Ratio and add it to the DataFrame
monthly_data['Sharpe_Ratio'] = monthly_data['Avg_Daily_Return'] / monthly_data['Std_Daily_Return']


In [4]:
print(monthly_data.head())

        Date       Open       High        Low      Close  Adj Close  \
0 2011-01-31  11.992857  12.144286  11.939286  12.118571  10.259578   
1 2011-02-28  12.544286  12.680357  12.540000  12.614643  10.679551   
2 2011-03-31  12.370000  12.492857  12.359286  12.446786  10.537444   
3 2011-04-30  12.385000  12.641071  12.381071  12.504643  10.586424   
4 2011-05-31  12.182143  12.422500  12.178571  12.422500  10.516882   

       Volume  Daily_Return  Dollar_Volume  Avg_Daily_Return  ...  \
0   377246800      0.051960   3.870393e+09          0.024222  ...   
1   403074000      0.040935   4.304649e+09          0.024222  ...   
2   274019200     -0.013306   2.887462e+09          0.024222  ...   
3  1006345200      0.004648   1.065360e+10          0.024222  ...   
4   417754400     -0.006569   4.393474e+09          0.024222  ...   

   Skewness_Daily_Return  Avg_Dollar_Volume  Std_Dollar_Volume  \
0              -0.082208       8.311242e+09       5.894129e+09   
1              -0.082208  

In [5]:
monthly_data.sort_values(by=['Symbol', 'Date'], inplace=True)


In [6]:
filtered_data = monthly_data[(monthly_data['Date'] >= '2010-01-01') & (monthly_data['Date'] <= '2023-12-31')]


Problem-1

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Load the data
data = pd.read_csv('monthly_data.csv', parse_dates=['Date'])
data.sort_values(by=['Symbol', 'Date'], inplace=True)

# Drop rows where the target or any predictors are NaN
data.dropna(subset=['Next_Month_Return'] + [col for col in data.columns if col != 'Next_Month_Return'], inplace=True)

# Splitting the dataset into training and testing sets
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns.tolist()
X = data[numeric_cols].drop(columns=['Next_Month_Return'])
y = data['Next_Month_Return']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

# Normalization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [10]:
# Define the model
ridge = Ridge()

# Define a range of regularization parameters to test
parameters = {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

# Setting up the grid search to find the best regularization parameter
ridge_regressor = GridSearchCV(ridge, parameters, scoring='r2', cv=5)
ridge_regressor.fit(X_train_scaled, y_train)

# Best regularization parameter
print("Best regularization parameter (alpha):", ridge_regressor.best_params_)


Best regularization parameter (alpha): {'alpha': 100.0}


In [11]:
# Predictions
y_train_pred = ridge_regressor.predict(X_train_scaled)
y_test_pred = ridge_regressor.predict(X_test_scaled)

# Calculating R-squared and MSE for the training set
train_r2 = r2_score(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)

# Calculating R-squared and MSE for the testing set
test_r2 = r2_score(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print("Training R-squared:", train_r2)
print("Training Mean Squared Error:", train_mse)
print("Testing R-squared:", test_r2)
print("Testing Mean Squared Error:", test_mse)


Training R-squared: 0.032862156401918585
Training Mean Squared Error: 0.005550480985205499
Testing R-squared: -0.07192183145781272
Testing Mean Squared Error: 0.005015988187866907


In [12]:
# Define a broader and more granular range of alpha values
alpha_range = [1e-3, 1e-2, 1e-1, 1, 10, 20, 50, 75, 100, 150, 200]

# Setup the grid search
ridge_regressor = GridSearchCV(ridge, {'alpha': alpha_range}, scoring='r2', cv=5)
ridge_regressor.fit(X_train_scaled, y_train)

# Print the best alpha found and corresponding R-squared
print("Best regularization parameter (alpha):", ridge_regressor.best_params_)
best_model = ridge_regressor.best_estimator_
print("Best CV R-squared:", ridge_regressor.best_score_)

# Reevaluate the model with the new alpha
y_train_pred = best_model.predict(X_train_scaled)
y_test_pred = best_model.predict(X_test_scaled)
print("Training R-squared:", r2_score(y_train, y_train_pred))
print("Testing R-squared:", r2_score(y_test, y_test_pred))


Best regularization parameter (alpha): {'alpha': 200}
Best CV R-squared: -0.03449180580924634
Training R-squared: 0.028640399027446417
Testing R-squared: -0.054963226881706184


Problem-2

In [13]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

# Assuming data is already loaded and split into X_train, X_test, y_train, y_test
# Normalize data (repeat from the previous setups if not already applied)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
dtree = DecisionTreeRegressor(random_state=8)

# Create the parameter grid
param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'min_samples_split': [2, 10, 20, 30],
    'max_leaf_nodes': [None, 10, 20, 30, 40, 50]
}

# Set up the grid search
grid_search = GridSearchCV(estimator=dtree, param_grid=param_grid, cv=5, scoring='r2')

# Fit the model
grid_search.fit(X_train_scaled, y_train)

# Output the best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation R-squared: ", grid_search.best_score_)

# Assess the best model
best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train_scaled)
y_test_pred = best_model.predict(X_test_scaled)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Print model performance
print("Training R-squared: ", train_r2)
print("Testing R-squared: ", test_r2)
print("Training MSE: ", train_mse)
print("Testing MSE: ", test_mse)


Best parameters found:  {'max_depth': None, 'max_leaf_nodes': 10, 'min_samples_leaf': 2, 'min_samples_split': 20}
Best cross-validation R-squared:  -0.15499717444972788
Training R-squared:  0.18818428901088036
Testing R-squared:  -0.22034690751547514
Training MSE:  0.004659074915911118
Testing MSE:  0.005710533635528854


In [15]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler

# Assuming data is already loaded and split into X_train, X_test, y_train, y_test
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the model
random_forest = RandomForestRegressor(random_state=8)

# Create a coarser parameter grid
param_grid = {
    'n_estimators': [100, 200],  # Reduced number of options
    'max_depth': [None, 20],  # Focused on potentially more impactful depths
    'min_samples_leaf': [1, 4],  # Coarser steps
    'min_samples_split': [2, 20],  # Covering extremes
    'max_leaf_nodes': [None, 50]  # Simplified to extreme and a middle point
}

# Set up the grid search with fewer folds
grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=3, scoring='r2', n_jobs=-1)

# Fit the model
grid_search.fit(X_train_scaled, y_train)

# Output the best parameters and best cross-validation score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation R-squared: ", grid_search.best_score_)

# Assess the best model
best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train_scaled)
y_test_pred = best_model.predict(X_test_scaled)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Print model performance
print("Training R-squared: ", train_r2)
print("Testing R-squared: ", test_r2)
print("Training MSE: ", train_mse)
print("Testing MSE: ", test_mse)


Best parameters found:  {'max_depth': None, 'max_leaf_nodes': None, 'min_samples_leaf': 4, 'min_samples_split': 20, 'n_estimators': 100}
Best cross-validation R-squared:  -0.07936488611891845
Training R-squared:  0.4663920996659332
Testing R-squared:  -0.1431584496583609
Training MSE:  0.003062418169204132
Testing MSE:  0.0053493352892610225


In [16]:
# Redefine the model with new parameters
random_forest_adjusted = RandomForestRegressor(
    n_estimators=200,  # increased from 100
    max_depth=30,       # a reasonable cap to prevent very deep trees
    min_samples_leaf=4,
    min_samples_split=20,
    max_leaf_nodes=None,  # consider setting a cap here if overfitting persists
    random_state=8
)

# Fit the model
random_forest_adjusted.fit(X_train_scaled, y_train)

# Evaluate the adjusted model
y_train_pred = random_forest_adjusted.predict(X_train_scaled)
y_test_pred = random_forest_adjusted.predict(X_test_scaled)
print("Adjusted Training R-squared: ", r2_score(y_train, y_train_pred))
print("Adjusted Testing R-squared: ", r2_score(y_test, y_test_pred))


Adjusted Training R-squared:  0.469623956438076
Adjusted Testing R-squared:  -0.15209188816492247


Problem-4

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error

nn_model = MLPRegressor(random_state=8)


In [19]:
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate_init': [0.001, 0.01],
    'max_iter': [200]  # You can adjust this depending on the convergence
}

grid_search = GridSearchCV(estimator=nn_model, param_grid=param_grid, cv=3, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Output the best parameters and best cross-validation score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation R-squared: ", grid_search.best_score_)


Best parameters found:  {'activation': 'relu', 'hidden_layer_sizes': (100, 50), 'learning_rate_init': 0.01, 'max_iter': 200, 'solver': 'adam'}
Best cross-validation R-squared:  -0.24791467491192995


In [20]:
# Assess the best model
best_nn_model = grid_search.best_estimator_
y_train_pred = best_nn_model.predict(X_train_scaled)
y_test_pred = best_nn_model.predict(X_test_scaled)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

# Print model performance
print("Training R-squared: ", train_r2)
print("Testing R-squared: ", test_r2)
print("Training MSE: ", train_mse)
print("Testing MSE: ", test_mse)


Training R-squared:  0.12418881335173626
Testing R-squared:  -0.20304085924163195
Training MSE:  0.005026350039241806
Testing MSE:  0.005629551113135226


In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Define the model
model = Sequential([
    Dense(100, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(50, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# Setup early stopping
early_stopping_monitor = EarlyStopping(patience=5)

# Fit the model
history = model.fit(X_train_scaled, y_train, validation_split=0.2, epochs=200,
                    callbacks=[early_stopping_monitor], verbose=1)

# Evaluate the model
train_loss, train_mse = model.evaluate(X_train_scaled, y_train, verbose=0)
test_loss, test_mse = model.evaluate(X_test_scaled, y_test, verbose=0)

print("Training MSE:", train_mse)
print("Testing MSE:", test_mse)


Epoch 1/200
13/13 [==============================] - 1s 21ms/step - loss: 0.0668 - mean_squared_error: 0.0668 - val_loss: 0.0154 - val_mean_squared_error: 0.0154
Epoch 2/200
13/13 [==============================] - 0s 6ms/step - loss: 0.0175 - mean_squared_error: 0.0175 - val_loss: 0.0134 - val_mean_squared_error: 0.0134
Epoch 3/200
13/13 [==============================] - 0s 6ms/step - loss: 0.0150 - mean_squared_error: 0.0150 - val_loss: 0.0104 - val_mean_squared_error: 0.0104
Epoch 4/200
13/13 [==============================] - 0s 6ms/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.0110 - val_mean_squared_error: 0.0110
Epoch 5/200
13/13 [==============================] - 0s 7ms/step - loss: 0.0067 - mean_squared_error: 0.0067 - val_loss: 0.0092 - val_mean_squared_error: 0.0092
Epoch 6/200
13/13 [==============================] - 0s 6ms/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.0088 - val_mean_squared_error: 0.0088
Epoch 7/200
13/13 [==============

Problem-6

In [24]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures

# Assuming 'data' is your original DataFrame
# Select numeric columns for interaction terms
numeric_cols = monthly_data.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Exclude the target variable from feature interactions
features = monthly_data[numeric_cols].drop(columns=['Next_Month_Return'])

# Generate polynomial and interaction features
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
features_poly = poly.fit_transform(features)
features_poly_df = pd.DataFrame(features_poly, columns=poly.get_feature_names_out(features.columns))

# Combine with original data
enhanced_data = pd.concat([monthly_data.drop(columns=numeric_cols), features_poly_df], axis=1)


In [37]:
enhanced_data = pd.concat([monthly_data[['Next_Month_Return']].reset_index(drop=True), features_poly_df], axis=1)

# Now check the columns to make sure 'Next_Month_Return' is included
print(enhanced_data.head())

   Next_Month_Return       Open       High        Low      Close  Adj Close  \
0           0.040935  11.992857  12.144286  11.939286  12.118571  10.259578   
1          -0.013306  12.544286  12.680357  12.540000  12.614643  10.679551   
2           0.004648  12.370000  12.492857  12.359286  12.446786  10.537444   
3          -0.006569  12.385000  12.641071  12.381071  12.504643  10.586424   
4          -0.034960  12.182143  12.422500  12.178571  12.422500  10.516882   

         Volume  Daily_Return  Dollar_Volume  Avg_Daily_Return  ...  \
0  3.772468e+08      0.051960   3.870393e+09          0.024222  ...   
1  4.030740e+08      0.040935   4.304649e+09          0.024222  ...   
2  2.740192e+08     -0.013306   2.887462e+09          0.024222  ...   
3  1.006345e+09      0.004648   1.065360e+10          0.024222  ...   
4  4.177544e+08     -0.006569   4.393474e+09          0.024222  ...   

   Cumulative_Return_6M^2  Cumulative_Return_6M Cumulative_Return_9M  \
0                0.101779 

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split the data
X = enhanced_data.drop(columns=['Next_Month_Return'])
y = enhanced_data['Next_Month_Return']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)


X_train = X_train[y_train.notnull()]
y_train = y_train[y_train.notnull()]

# Since X_train was likely transformed with a scaler, make sure to apply transformations again if necessary
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)



In [33]:
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error

# Define and train the Ridge Regression model
best_alpha = 100
ridge = Ridge(alpha=best_alpha)  # Use the best alpha found earlier or tune again if needed
ridge.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred = ridge.predict(X_train_scaled)
y_test_pred = ridge.predict(X_test_scaled)

# Evaluate the model
print("Training R-squared:", r2_score(y_train, y_train_pred))
print("Testing R-squared:", r2_score(y_test, y_test_pred))
print("Training MSE:", mean_squared_error(y_train, y_train_pred))
print("Testing MSE:", mean_squared_error(y_test, y_test_pred))



Training R-squared: 0.10995697982751274
Testing R-squared: -0.027278345639673462
Training MSE: 0.004921935214873224
Testing MSE: 0.005684073934259992


Problem-7

In [45]:
import pandas as pd

# Define parameters for financial metrics (make sure this matches your actual use)
params = {
    'profitability': ('NetIncomeLoss', 'Assets'),
    'liquidity': ('AssetsCurrent', 'LiabilitiesCurrent'),
    'activity': ('Revenues', 'Assets'),
    'financial_leverage': ('Liabilities', 'Assets')
}

def process_financial_data(data, params):
    results = []
    if not data or 'facts' not in data or 'us-gaap' not in data['facts']:
        return results  # Ensure we're returning a list even if empty

    us_gaap = data['facts']['us-gaap']
    for key, (num, den) in params.items():
        num_data = us_gaap.get(num, {}).get('units', {}).get('USD', [])
        den_data = us_gaap.get(den, {}).get('units', {}).get('USD', [])
        if num_data and den_data:
            num_df = pd.DataFrame(num_data)
            den_df = pd.DataFrame(den_data)
            num_df['end'] = pd.to_datetime(num_df['end'])
            den_df['end'] = pd.to_datetime(den_df['end'])
            merged_df = pd.merge(num_df, den_df, on='end', suffixes=('_num', '_den'))
            merged_df[key] = merged_df['val_num'] / merged_df['val_den']
            results.append(merged_df[['end', key]])
        else:
            print(f"Data for {key} not found or incomplete.")

    return results

# Main execution loop to fetch and process data for all companies
companies = {'AAPL': '0000320193', 'MSFT': '0000789019', 'GOOGL': '0001652044', 'AMZN': '0001018724'}
all_financials = pd.DataFrame()
for ticker, cik in companies.items():
    raw_data = fetch_financial_data(cik)
    financials = process_financial_data(raw_data, params)
    for df in financials:
        if isinstance(df, pd.DataFrame):  # Check if df is a DataFrame
            df['ticker'] = ticker  # Assign ticker
            all_financials = pd.concat([all_financials, df], ignore_index=True)
        else:
            print(f"Expected a DataFrame, got {type(df)} instead.")


Data for activity not found or incomplete.
Data for financial_leverage not found or incomplete.


In [4]:
import requests
import pandas as pd

# Define headers to access the SEC API
headers = {'User-Agent': 'your_email@example.com'}  # Replace with your email

# Companies and their CIK numbers
companies = {
    'AAPL': '0000320193',
    'MSFT': '0000789019',
    'GOOGL': '0001652044',
    'AMZN': '0001018724'
}

# Financial parameters to calculate ratios
params = {
    'profitability': ('NetIncomeLoss', 'Assets'),
    'liquidity': ('AssetsCurrent', 'LiabilitiesCurrent'),
    'activity': ('Revenues', 'Assets'),
    'financial_leverage': ('Liabilities', 'Assets')
}

def fetch_financial_data(cik):
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    return None

def process_financial_data(data, params):
    results = {}
    if not data or 'facts' not in data or 'us-gaap' not in data['facts']:
        return results

    us_gaap = data['facts']['us-gaap']
    for key, (num, den) in params.items():
        num_data = us_gaap.get(num, {}).get('units', {}).get('USD', [])
        den_data = us_gaap.get(den, {}).get('units', {}).get('USD', [])

        if num_data and den_data:
            num_df = pd.DataFrame(num_data).rename(columns={'val': 'num_val'})
            den_df = pd.DataFrame(den_data).rename(columns={'val': 'den_val'})

            num_df['end'] = pd.to_datetime(num_df['end'])
            den_df['end'] = pd.to_datetime(den_df['end'])

            merged_df = pd.merge(num_df, den_df, on='end', suffixes=('_num', '_den'))
            merged_df[key] = merged_df['num_val'] / merged_df['den_val']
            results[key] = merged_df[['end', key]]

    return results

# Fetch and process data for all companies
for company, cik in companies.items():
    data = fetch_financial_data(cik)
    if data:
        processed_data = process_financial_data(data, params)
        # Save each company's data to CSV
        for ratio_name, df in processed_data.items():
            df.to_csv(f"{company}_{ratio_name}.csv", index=False)
    else:
        print(f"Failed to fetch data for {company}")


In [1]:
import pandas as pd

# Define the directory and the scope of companies and ratios
directory = './'
companies = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']
ratios = ['profitability', 'liquidity', 'activity', 'financial_leverage']

# Function to process a single company's ratios
def process_company_ratios(company):
    consolidated_ratios = pd.DataFrame()

    for ratio in ratios:
        file_path = f"{directory}{company}_{ratio}.csv"
        try:
            print(f"Processing {file_path}...")
            ratio_data = pd.read_csv(file_path)
            ratio_data['end'] = pd.to_datetime(ratio_data['end'])

            # Filtering data within the date range and handling data properly
            filtered_data = ratio_data[(ratio_data['end'] >= '2010-01-01') & (ratio_data['end'] <= '2023-12-31')].copy()
            if not filtered_data.empty:
                filtered_data.rename(columns={'end': 'Date', ratio_data.columns[1]: f'{company}_{ratio}'}, inplace=True)

                if consolidated_ratios.empty:
                    consolidated_ratios = filtered_data[['Date', f'{company}_{ratio}']]
                else:
                    consolidated_ratios = pd.merge(consolidated_ratios, filtered_data[['Date', f'{company}_{ratio}']], on='Date', how='outer')

        except FileNotFoundError:
            print(f"File not found: {file_path}")
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")

    return consolidated_ratios

# Process each company one at a time
for company in companies:
    company_data = process_company_ratios(company)
    if not company_data.empty:
        company_data.sort_values(by='Date', inplace=True)
        company_data.to_csv(f"{directory}consolidated_{company}_financial_ratios.csv", index=False)
        print(f"Consolidated financial ratios for {company} saved successfully.")
    else:
        print(f"No data was consolidated for {company}.")


Processing ./AAPL_profitability.csv...
Processing ./AAPL_liquidity.csv...
Processing ./AAPL_activity.csv...
Processing ./AAPL_financial_leverage.csv...
Consolidated financial ratios for AAPL saved successfully.
Processing ./GOOGL_profitability.csv...
Processing ./GOOGL_liquidity.csv...
Processing ./GOOGL_activity.csv...
Processing ./GOOGL_financial_leverage.csv...
Consolidated financial ratios for GOOGL saved successfully.
Processing ./MSFT_profitability.csv...
Processing ./MSFT_liquidity.csv...
Processing ./MSFT_activity.csv...
Processing ./MSFT_financial_leverage.csv...
Consolidated financial ratios for MSFT saved successfully.
Processing ./AMZN_profitability.csv...
Processing ./AMZN_liquidity.csv...
Processing ./AMZN_activity.csv...
File not found: ./AMZN_activity.csv
Processing ./AMZN_financial_leverage.csv...
File not found: ./AMZN_financial_leverage.csv
Consolidated financial ratios for AMZN saved successfully.


In [2]:
import pandas as pd

# Function to merge ratios for a specific company
def merge_company_data(company):
    # Load the monthly data
    monthly_data = pd.read_csv('monthly_data.csv')
    monthly_data['Date'] = pd.to_datetime(monthly_data['Date'])

    try:
        # Load the consolidated financial ratios for the company
        company_ratios = pd.read_csv(f'./consolidated_{company}_financial_ratios.csv')
        company_ratios['Date'] = pd.to_datetime(company_ratios['Date'])

        # Merge with the monthly data on 'Date'
        merged_data = pd.merge(monthly_data, company_ratios, on='Date', how='left')
        print(f"{company} ratios merged successfully.")

        # Save the merged dataset
        merged_data.to_csv(f'fully_merged_monthly_data_{company}.csv', index=False)
        print(f"Merged data for {company} saved successfully.")
    except FileNotFoundError:
        print(f"Consolidated ratio file for {company} not found. Skipping.")
    except Exception as e:
        print(f"Error while merging data for {company}: {str(e)}")

# List of companies
companies = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']

# Call the function for each company
for company in companies:
    merge_company_data(company)


AAPL ratios merged successfully.
Merged data for AAPL saved successfully.
GOOGL ratios merged successfully.
Merged data for GOOGL saved successfully.
MSFT ratios merged successfully.
Merged data for MSFT saved successfully.
AMZN ratios merged successfully.
Merged data for AMZN saved successfully.


In [3]:
import pandas as pd

# List of companies
companies = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']

# Initialize an empty DataFrame to hold all data
all_data = pd.DataFrame()

# Iterate through each company and append their data
for company in companies:
    file_path = f'fully_merged_monthly_data_{company}.csv'
    company_data = pd.read_csv(file_path)
    all_data = pd.concat([all_data, company_data], ignore_index=True)

# Optionally, you can sort by date if it's necessary for your analysis
all_data.sort_values(by='Date', inplace=True)

# Save the combined dataset
all_data.to_csv('fully_merged_monthly_data_all_companies.csv', index=False)
print("All company data merged into a single file successfully.")


All company data merged into a single file successfully.


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load the consolidated data
data = pd.read_csv('fully_merged_monthly_data_all_companies.csv')
data['Date'] = pd.to_datetime(data['Date'])

# Prepare features and target
# Assuming 'Next_Month_Return' is the target variable and the rest are features
# You may need to adjust these according to your actual data columns
X = data.drop(['Date', 'Next_Month_Return'], axis=1)
y = data['Next_Month_Return']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define and train the Ridge Regression model
best_alpha = 100  # Use the best alpha found earlier or tune again if needed
ridge = Ridge(alpha=best_alpha)
ridge.fit(X_train_scaled, y_train)

# Make predictions
y_train_pred = ridge.predict(X_train_scaled)
y_test_pred = ridge.predict(X_test_scaled)

# Evaluate the model
print("Training R-squared:", r2_score(y_train, y_train_pred))
print("Testing R-squared:", r2_score(y_test, y_test_pred))
print("Training MSE:", mean_squared_error(y_train, y_train_pred))
print("Testing MSE:", mean_squared_error(y_test, y_test_pred))


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the consolidated data
data = pd.read_csv('fully_merged_monthly_data_all_companies.csv')
data['Date'] = pd.to_datetime(data['Date'])

# Strategy 1: Filter based on a more recent date range, e.g., last 5 years
filtered_data = data[data['Date'] >= pd.to_datetime("2018-01-01")]

# Strategy 3: Random sampling - sample 50% of the data
sampled_data = filtered_data.sample(frac=0.5, random_state=8)

# Proceed with the sampled data
X = sampled_data.drop(['Date', 'Next_Month_Return'], axis=1)
y = sampled_data['Next_Month_Return']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)

# Continue with feature scaling and model training as before...


KeyboardInterrupt: 